In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n---------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------
loss: 2.301013 [    0/60000]
loss: 2.291429 [ 6400/60000]
loss: 2.267442 [12800/60000]
loss: 2.267129 [19200/60000]
loss: 2.234087 [25600/60000]
loss: 2.215422 [32000/60000]
loss: 2.220229 [38400/60000]
loss: 2.182964 [44800/60000]
loss: 2.186705 [51200/60000]
loss: 2.151971 [57600/60000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.142096

Epoch 2
---------
loss: 2.155917 [    0/60000]
loss: 2.151504 [ 6400/60000]
loss: 2.084016 [12800/60000]
loss: 2.105947 [19200/60000]
loss: 2.045479 [25600/60000]
loss: 1.988963 [32000/60000]
loss: 2.020272 [38400/60000]
loss: 1.937179 [44800/60000]
loss: 1.945104 [51200/60000]
loss: 1.873049 [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.867487

Epoch 3
---------
loss: 1.908548 [    0/60000]
loss: 1.882309 [ 6400/60000]
loss: 1.750311 [12800/60000]
loss: 1.797083 [19200/60000]
loss: 1.681195 [25600/60000]
loss: 1.639270 [32000/60000]
loss: 1.661623 [38400/60000]
loss: 1.559999 [44800/60000]
loss: 1.585069 [51200/60000]
los